In [1]:
import pandas as pd

In [3]:
%%time
df = pd.read_parquet("../data/raw/competition_data_final_pqt/")

CPU times: user 2min 47s, sys: 2min 34s, total: 5min 22s
Wall time: 1min 30s


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322899435 entries, 0 to 322899434
Data columns (total 12 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   region_name            object 
 1   city_name              object 
 2   cpe_manufacturer_name  object 
 3   cpe_model_name         object 
 4   url_host               object 
 5   cpe_type_cd            object 
 6   cpe_model_os_type      object 
 7   price                  float64
 8   date                   object 
 9   part_of_day            object 
 10  request_cnt            int64  
 11  user_id                int64  
dtypes: float64(1), int64(2), object(9)
memory usage: 28.9+ GB


In [21]:
df["user_id"].min()

0

In [22]:
df["user_id"].max()

415316

In [5]:
%%time
df = df.astype(
    {
        # "region_name": "category",
        # "city_name": "category",
        # "cpe_manufacturer_name": "category",
        # "cpe_model_name": "category",
        # "url_host": "category",
        # "cpe_type_cd": "category",
        # "cpe_model_os_type": "category",
        "price": "float32",
        "date": "datetime64[ns]",
        # "part_of_day": "category",
        "request_cnt": "uint8",
        "user_id": "int32"
    }
)

CPU times: user 42.6 s, sys: 38.2 s, total: 1min 20s
Wall time: 1min 11s


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322899435 entries, 0 to 322899434
Data columns (total 12 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   region_name            object        
 1   city_name              object        
 2   cpe_manufacturer_name  object        
 3   cpe_model_name         object        
 4   url_host               object        
 5   cpe_type_cd            object        
 6   cpe_model_os_type      object        
 7   price                  float32       
 8   date                   datetime64[ns]
 9   part_of_day            object        
 10  request_cnt            uint8         
 11  user_id                int32         
dtypes: datetime64[ns](1), float32(1), int32(1), object(8), uint8(1)
memory usage: 24.4+ GB


In [7]:
%%time
df.to_parquet("../data/processed/sessions-noncat.pq")

CPU times: user 3min 25s, sys: 41.9 s, total: 4min 7s
Wall time: 4min 3s


In [8]:
! ls -lh ../data/processed/

total 2.9G
-rw-rw-r-- 1 ababkin ababkin 1.7G Feb 18 16:30 sessions-noncat.pq
-rw-rw-r-- 1 ababkin ababkin 1.2G Feb 17 10:49 sessions.pq
-rw-rw-r-- 1 ababkin ababkin 1.7M Feb 17 11:30 test-split.csv
-rw-rw-r-- 1 ababkin ababkin 885K Feb 17 19:25 test-users.pq
-rw-rw-r-- 1 ababkin ababkin 2.0M Feb 17 19:24 train-users.pq


In [28]:
target = pd.read_parquet("../data/raw/public_train.pqt")

In [36]:
(target.index == target["user_id"]).all()

True

In [33]:
test = pd.read_parquet("../data/raw/submit_2.pqt")

In [51]:
target[target["age"].isna()]

,age,is_male,user_id
220333,NaN,1,220333


In [39]:
test[test.index != test["user_id"]]

,user_id
0,6
4,11
8,19
10,27
11,32
...,...
145307,415284
145308,415285
145309,415286
145312,415306


In [91]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322899435 entries, 0 to 322899434
Data columns (total 12 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   region_name            category      
 1   city_name              category      
 2   cpe_manufacturer_name  category      
 3   cpe_model_name         category      
 4   url_host               category      
 5   cpe_type_cd            category      
 6   cpe_model_os_type      category      
 7   price                  float32       
 8   date                   datetime64[ns]
 9   part_of_day            category      
 10  request_cnt            uint8         
 11  user_id                int32         
dtypes: category(8), datetime64[ns](1), float32(1), int32(1), uint8(1)
memory usage: 9.0 GB


In [66]:
import bisect
import numpy as np

In [67]:
target["age_bucket"] = target["age"].map(lambda x: bisect.bisect_left([18,25,35,45,55,65], x))
target.loc[target["age"].isna(), "age_bucket"] = np.nan

In [68]:
target[target["age"].isna()]

,age,is_male,user_id,age_bucket
220333,NaN,1,220333,NaN


In [74]:
target["is_male"] = target["is_male"].replace("NA", np.nan).astype(float)

In [77]:
(target["age"].dropna() == target["age"].dropna().astype(int)).all()

True

In [89]:
target = target.astype(
    {
        "age": "Int32",
        "is_male": "UInt8",
        "user_id": "int32",
        "age_bucket": "UInt8",
    }
)

In [92]:
target.loc[:, ["user_id", "is_male", "age_bucket", "age"]]

,user_id,is_male,age_bucket,age
350459,350459,1,2,31
188276,188276,1,2,35
99002,99002,0,3,41
155506,155506,0,2,33
213873,213873,0,4,54
...,...,...,...,...
225374,225374,1,4,49
25776,25776,1,1,22
148131,148131,0,2,28
205570,205570,1,2,28


In [145]:
target.to_parquet("../data/processed/train-users.pq", index=False)

In [150]:
test.astype("int32").to_parquet("../data/processed/test-users.pq", index=False)

In [118]:
htest = set(test[test.index == test["user_id"]]["user_id"])

In [119]:
subm = pd.read_csv("../data/raw/sample_submission.csv")

In [120]:
subm.loc[subm["user_id"].isin(htest), ["age", "is_male"]] = 0

In [132]:
subm["is_private"] = False

In [133]:
subm.loc[(subm["age"] == 0) & (subm["is_male"] == 0), "is_private"] = True

In [136]:
subm[["user_id", "is_private"]].to_csv("../data/processed/test-split.csv", index=False)

In [128]:
pd.Series(list(htest), name="user_id")

0        131076
1        393220
2             7
3        262153
4            10
          ...  
64723    131057
64724    131061
64725    131062
64726    262137
64727    131071
Name: user_id, Length: 64728, dtype: int64

In [121]:
subm.to_csv("../submissions/test-private-2.csv", index=False)

In [98]:
79996 / (64728 + 79996)

0.5527486802465382

In [99]:
(target["user_id"] == target.index).all()

True

In [140]:
subm[~subm["is_private"]]

,user_id,age,is_male,is_private
0,6,2,0.330467,False
1,11,5,0.725477,False
2,19,1,0.240190,False
3,27,2,0.536798,False
4,32,3,0.471325,False
...,...,...,...,...
79991,415284,2,0.510685,False
79992,415285,3,0.514142,False
79993,415286,3,0.536682,False
79994,415306,3,0.511746,False


In [141]:
df[df["user_id"] == 11]

,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt,user_id
60819777,Приморский край,Большой Камень,Huawei,P Smart Z,adwile.com,smartphone,Android,14190.0,2021-08-04,morning,1,11
60819778,Приморский край,Большой Камень,Huawei,P Smart Z,avatars.mds.yandex.net,smartphone,Android,14190.0,2021-08-06,day,2,11
60819779,Приморский край,Большой Камень,Huawei,P Smart Z,mail.yandex.ru,smartphone,Android,14190.0,2021-07-01,evening,1,11
60819780,Приморский край,Большой Камень,Huawei,P Smart Z,rueconomics-ru.turbopages.org,smartphone,Android,14190.0,2021-07-14,morning,1,11
60819781,Приморский край,Владивосток,Huawei,P Smart Z,googleads.g.doubleclick.net,smartphone,Android,14190.0,2021-07-26,day,1,11
...,...,...,...,...,...,...,...,...,...,...,...,...
60820343,Приморский край,Большой Камень,Huawei,P Smart Z,googleads.g.doubleclick.net,smartphone,Android,14190.0,2021-07-29,day,1,11
60820344,Приморский край,Большой Камень,Huawei,P Smart Z,rueconomics-ru.turbopages.org,smartphone,Android,14190.0,2021-07-08,morning,1,11
60820345,Приморский край,Большой Камень,Huawei,P Smart Z,ads.adfox.ru,smartphone,Android,14190.0,2021-08-02,morning,1,11
60820346,Приморский край,Большой Камень,Huawei,P Smart Z,avatars.mds.yandex.net,smartphone,Android,14190.0,2021-06-30,day,1,11
